# Introduction to connecting and Querrying the Augur DB

If you made to this point, welcome! :) This short tutorial will show how to connect to the database and how to do a simple querry. If you need the config file please email cdolfi@redhat.com

All the notebooks in this folder are based on https://github.com/chaoss/augur-community-reports templates. 

## Connect to your database

Until the Operate First enviroment can connect to the DB, use config file to access. Do not push config file to Github repo

In [4]:
import psycopg2
import pandas as pd 
import sqlalchemy as salc
import json
import os

with open("../config_temp.json") as config_file:
    config = json.load(config_file)

In [5]:
database_connection_string = 'postgresql+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = salc.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

### Retrieve Available Respositories

In [13]:
repolist = pd.DataFrame()
repo_query = salc.sql.text(f"""
             SET SCHEMA 'augur_data';
             SELECT a.rg_name,
                a.repo_group_id,
                b.repo_name,
                b.repo_id,
                b.forked_from,
                b.repo_archived,
                b.repo_git
            FROM
                repo_groups a,
                repo b
            WHERE
                a.repo_group_id = b.repo_group_id
            ORDER BY
                rg_name,
                repo_name;
    """)
repolist = pd.read_sql(repo_query, con=engine)
display(repolist)
repolist.dtypes

,rg_name,repo_group_id,repo_name,repo_id,forked_from,repo_archived,repo_git,repo_path
0,chaoss,25155,augur,25440,Parent not available,0,https://github.com/chaoss/augur,github.com/chaoss/
1,chaoss,25155,augur-license,25460,Parent not available,0,https://github.com/chaoss/augur-license,github.com/chaoss/
2,chaoss,25155,community-reports,25558,Parent not available,0,https://github.com/chaoss/community-reports,github.com/chaoss
3,chaoss,25155,governance,25449,Parent not available,0,https://github.com/chaoss/governance,github.com/chaoss/
4,chaoss,25155,grimoirelab,25448,Parent not available,0,https://github.com/chaoss/grimoirelab,github.com/chaoss/
...,...,...,...,...,...,...,...,...
75,SaltStack,60017,salt-winrepo-ng,30356,Parent not available,0,https://github.com/saltstack/salt-winrepo-ng,github.com/saltstack/
76,SaltStack,60017,salty-vagrant,30362,Parent not available,0,https://github.com/saltstack/salty-vagrant,github.com/saltstack/
77,SaltStack,60017,sublime-text,30347,Parent not available,0,https://github.com/saltstack/sublime-text,github.com/saltstack/
78,SaltStack,60017,takara,30380,Parent not available,1,https://github.com/saltstack/takara,github.com/saltstack/


rg_name          object
repo_group_id     int64
repo_name        object
repo_id           int64
forked_from      object
repo_archived     int64
repo_git         object
repo_path        object
dtype: object

### Create a Simpler List for quickly Identifying repo_group_id's and repo_id's for other queries

In [15]:

repolist = pd.DataFrame()

repo_query = salc.sql.text(f"""
             SET SCHEMA 'augur_data';
             SELECT b.repo_id,
                a.repo_group_id,
                b.repo_name,
                a.rg_name,
                b.repo_git
            FROM
                repo_groups a,
                repo b 
            WHERE
                a.repo_group_id = b.repo_group_id 
            ORDER BY
                rg_name,
                repo_name;   

    """)

repolist = pd.read_sql(repo_query, con=engine)

repolist

,repo_id,repo_group_id,repo_name,rg_name,repo_git
0,25440,25155,augur,chaoss,https://github.com/chaoss/augur
1,25460,25155,augur-license,chaoss,https://github.com/chaoss/augur-license
2,25558,25155,community-reports,chaoss,https://github.com/chaoss/community-reports
3,25449,25155,governance,chaoss,https://github.com/chaoss/governance
4,25448,25155,grimoirelab,chaoss,https://github.com/chaoss/grimoirelab
...,...,...,...,...,...
75,30356,60017,salt-winrepo-ng,SaltStack,https://github.com/saltstack/salt-winrepo-ng
76,30362,60017,salty-vagrant,SaltStack,https://github.com/saltstack/salty-vagrant
77,30347,60017,sublime-text,SaltStack,https://github.com/saltstack/sublime-text
78,30380,60017,takara,SaltStack,https://github.com/saltstack/takara


### Create a list of all of the tables with the total number of data entries 

In [16]:
repolist = pd.DataFrame()

repo_query = salc.sql.text(f"""
                ANALYZE;
                SELECT schemaname,relname,n_live_tup 
                  FROM pg_stat_user_tables 
                  ORDER BY n_live_tup DESC;

    """)

repolist = pd.read_sql(repo_query, con=engine)

repolist

,schemaname,relname,n_live_tup
0,augur_data,commits,320271
1,augur_data,repo_labor,312000
2,augur_data,pull_request_files,149825
3,augur_data,message,93410
4,augur_data,issue_events,79373
...,...,...,...
174,augur_data,pull_request_analysis,0
175,augur_data,message_sentiment_summary,0
176,augur_data,message_sentiment,0
177,augur_data,message_analysis_summary,0


Congrats you have done your first queries! There will be a few more simple examples below on how to pull an entire table. If you would like to explore on your own, the schema.png on the home sandiego directory will be greatly helpful in your explorations! Happy querying :) 

### Data from the messages 

In [17]:
repolist = pd.DataFrame()

repo_query = salc.sql.text(f"""
             SET SCHEMA 'augur_data';
             SELECT * FROM message
    """)

repolist = pd.read_sql(repo_query, con=engine)

repolist

,msg_id,rgls_id,msg_text,msg_timestamp,msg_sender_email,msg_header,pltfrm_id,tool_source,tool_version,data_source,data_collection_date,cntrb_id
0,1715557,None,All of these are repeated. Why don't you creat...,2015-12-17 22:54:17,None,None,25150,GitHub API Worker,1.0.0,GitHub API,NaT,279368
1,1715697,None,"@acs, @valeriocos:\r\n\r\nMappings should foll...",2018-02-28 20:08:50,None,None,25150,GitHub API Worker,1.0.0,GitHub API,NaT,279373
2,1691709,None,All of these are repeated. Why don't you creat...,2015-12-17 22:54:17,None,None,25150,GitHub API Worker,0.0.3,GitHub API,NaT,277140
3,1691710,None,this change does not seem related to the purpo...,2015-12-17 22:57:10,None,None,25150,GitHub API Worker,0.0.3,GitHub API,NaT,277140
4,1691711,None,"I think that function like this, and `read_fi...",2015-12-17 23:07:08,None,None,25150,GitHub API Worker,0.0.3,GitHub API,NaT,277140
...,...,...,...,...,...,...,...,...,...,...,...,...
93405,1785136,None,"I see! Yes. The only problem here is, what if ...",2013-04-02 14:23:34,None,None,25150,GitHub Pull Request Worker,1.0.0,GitHub API,2021-08-17 21:16:34,280738
93406,1785137,None,"Ya, that would an issue if there are multiple ...",2013-04-02 14:36:33,None,None,25150,GitHub Pull Request Worker,1.0.0,GitHub API,2021-08-17 21:16:34,280725
93407,1785138,None,Not sure how you can do the math if you don't ...,2013-04-02 14:43:21,None,None,25150,GitHub Pull Request Worker,1.0.0,GitHub API,2021-08-17 21:16:34,280738
93408,1785139,None,Currently though the: _x unknown yet to accept...,2013-04-02 14:58:46,None,None,25150,GitHub Pull Request Worker,1.0.0,GitHub API,2021-08-17 21:16:34,280725


### Contributor affiliation data

In [18]:
repolist = pd.DataFrame()

repo_query = salc.sql.text(f"""
             SET SCHEMA 'augur_data';
             SELECT * FROM contributor_affiliations
    """)

repolist = pd.read_sql(repo_query, con=engine)

repolist

,ca_id,ca_domain,ca_start_date,tool_source,tool_version,data_source,data_collection_date,ca_last_used,ca_affiliation,ca_active
0,25152,goggins.com,2016-03-03,None,None,None,2019-10-29 17:22:30,2020-04-17 21:06:25,None,1
1,1,samsung.com,1970-01-01,load,1.0,load,1970-01-01 00:00:00,2018-08-01 18:37:54,Samsung,1
2,2,linuxfoundation.org,1970-01-01,load,1.0,load,1970-01-01 00:00:00,2018-08-01 18:37:54,Linux Foundation,1
3,3,ibm.com,1970-01-01,load,1.0,load,1970-01-01 00:00:00,2018-08-01 18:37:54,IBM,1
4,8,walmart.com,1970-01-01,load,1.0,load,1970-01-01 00:00:00,2018-09-01 06:00:00,Walmart,1
...,...,...,...,...,...,...,...,...,...,...
703,25325,dgarnier@pivotal.io,1970-01-01,Helper Script,None,Dawn's vmware_mapping JSON,2020-04-28 18:52:49,2020-04-28 18:52:49,VMware,1
704,25326,git@garnier.wf,1970-01-01,Helper Script,None,Dawn's vmware_mapping JSON,2020-04-28 18:52:49,2020-04-28 18:52:49,VMware,1
705,25327,matthew.heidemann@gmail.com,1970-01-01,Helper Script,None,Dawn's vmware_mapping JSON,2020-04-28 18:52:49,2020-04-28 18:52:49,VMware,1
706,25328,me@lurraca.com,1970-01-01,Helper Script,None,Dawn's vmware_mapping JSON,2020-04-28 18:52:49,2020-04-28 18:52:49,VMware,1
